# Redis - Simple Social Network Datastore Example

This source code contains example of redis command based on the datastructure of simple social network.

In [1]:
import redis, csv

In [2]:
# connect to the local redis server

rd = redis.Redis()

In [3]:
# remove all data so that it is clear if we rerun this notebook

rd.flushall()

True

## Import data
Read data from csv files and store all data (user information and relationship) in the redis

In [4]:
with open('users.csv', encoding='utf-8') as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        id = row[0]
        name = row[1]
        k = 'user:{}:name'.format(id)
        rd.set(k, name)
        k = 'username:{}'.format(name)
        rd.set(k, id)

In [5]:
with open('follows.csv', encoding='utf-8') as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        id = row[0]
        follow = row[1]
        k = 'user:{}:follows'.format(id)
        rd.sadd(k, follow)
        k = 'user:{}:followed_by'.format(follow)
        rd.sadd(k, id)

## Data Retrieval
To retrieve data from redis, we will have to know the data structure being used for that particular key and use command accordingly.  In this case, user information is stored as "String" type and following information is stored as "Set"

In [6]:
id = 141
name = rd.get('user:{}:name'.format(id)).decode('utf-8')
name

'crushedGnu1'

In [7]:
rd.get('username:{}'.format(name)).decode('utf-8')

'141'

In [8]:
rd.smembers('user:{}:follows'.format(id))

{b'106', b'108', b'119', b'138', b'143', b'162', b'164', b'180'}

In [9]:
rd.smembers('user:{}:followed_by'.format(id))

{b'106', b'152', b'187'}

## Key scan
We can scan all keys in the database using "scan" command.  We call scan with cursor (initial at 0) to indicate the place to start scanning.  We also indicate the matching pattern to focus our scan to specific key pattern.  Each call returns a tuple of (next_cursor, results).  We can use next_cursor to continue our scanning process.

In [13]:
cursor = 0
cursor, keys = rd.scan(cursor=cursor, match='username:*')
while cursor > 0:
    for x in keys:
        key = x.decode('utf-8')
        value = rd.get(key).decode('utf-8')
        print(key, value)
        
    cursor, keys = rd.scan(cursor=cursor, match='username:*')  

username:crushedGnu1 141
username:ferventCur8 142
username:pitifulSalami5 134
username:ecstaticMallard3 168
username:bubblyWigeon3 183
username:importedCoati3 133
username:euphoricTortoise4 143
username:ardentEland9 162
username:solemnRelish2 174
username:solemnBoars0 179
username:pitifulShads3 116
username:abjectBuck4 146
username:dreadfulAbalone4 110
username:ecstaticMandrill3 128
username:brainyBoars3 112
username:giddyBass5 102
username:dejectedOil5 157
username:cautiousTuna9 195
username:madToucan7 178
username:mercifulHoopoe6 177
username:peskyCaviar0 109
username:outlyingBustard9 115
username:debonairBobolink7 125
username:shySnipe9 130
username:wearyBasmati1 180
username:giddyStork2 182
username:dreadfulToucan1 120
username:lyingPoultry2 121
username:mellowCordial5 186
username:debonairBagels9 150
username:hushedPretzels3 158
username:fondGatorade0 106
username:peskyMoth8 127
username:enragedDoves7 176
username:jumpySheep0 113
username:guiltyMeerkat7 138
username:trustingIguana

In [10]:
rd.scan(0)

(224,
 [b'username:crushedGnu1',
  b'user:121:followed_by',
  b'username:ferventCur8',
  b'user:181:follows',
  b'user:179:follows',
  b'user:148:followed_by',
  b'username:pitifulSalami5',
  b'user:200:followed_by',
  b'user:165:name',
  b'user:163:followed_by'])

In [12]:
rd.scan(224)

(48,
 [b'user:147:follows',
  b'username:ecstaticMallard3',
  b'username:bubblyWigeon3',
  b'username:importedCoati3',
  b'user:168:follows',
  b'username:euphoricTortoise4',
  b'user:157:follows',
  b'user:145:followed_by',
  b'user:113:name',
  b'user:110:followed_by'])